[Reference](https://pub.towardsai.net/building-a-spicy-pepper-classifier-with-no-datasets-96-accuracy-8262d54a8117)

In [1]:
#   measurements
pepper_measurements_px = [
                          ['Anaheim', 262, 63, 'Green'],
                          ['Cubanelle', 222, 70, 'Green'],
                          ['Cayenne', 249, 22, 'Red'],
                          ['Shishito', 140, 21, 'Green'],
                          ['Hungarian Wax', 148, 63, 'Orange'],
['Jimmy Nardello', 190, 23, 'Red'],
                          ['Fresno', 120, 43, 'Red'],
                          ['Jalapeno', 106, 40, 'Dark Green'],
                          ['Aji Amarillo', 92, 13, 'Yellow'],
                          ['Aji Dulce', 81, 30, 'Red'],
['Serrano', 74, 14, 'Dark Green'],
                          ['Padron', 62, 38, 'Dark Green'],
                          ['Scotch Bonnet', 37, 42, 'Yellow'],
                          ['Habanero', 67, 21, 'Orange'],
                          ['Cumari', 18, 11, 'Yellow'],
]

In [3]:
#simulated probability distribution of one stock
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def create_peppers(sd, mean, alfa, size):
  #invertire il segno di alfa
  x = skewnorm.rvs(-alfa, size=size) 
  def calc(k, sd, mean):
    return (k*sd)+mean
  x = calc(x, sd, mean) #standard distribution
  
  #graph the distribution
  #pd.DataFrame(x).hist(bins=100)
  #pick one random number from the distribution
  #formally I would use cdf, but I just have to pick randomly from the 1000000 samples
  df = [np.random.choice(x) for k in range(size)]
  #return the DataFrame
  return pd.DataFrame(df)
  
def cm_converter(px_measurements):
  pc_cm = 0.05725
  for _ in range(len(px_measurements)):
    px_measurements[_][1] *= pc_cm
    px_measurements[_][2] *= pc_cm
  return px_measurements

In [4]:
#   create converted list
pepper_measurements_cm = cm_converter(pepper_measurements_px)

#   create final datasets
heigh_sd = 0.1
width_sd = 0.1

df = pd.DataFrame()
for _ in pepper_measurements_cm:
  #   create height
  #SD is 10% of the height
  df_height = create_peppers(_[1]*heigh_sd, _[1], 0, 100000)
  #   create width
  #SD is 10% of the width
  df_width = create_peppers(_[2]*width_sd, _[2], 0, 100000)
  #create DataFrame
  df_single = pd.concat([df_height, df_width], axis=1)
  df_single.columns = ['height', 'width']
  #create name
  df_single['name'] = str(_[0])
  df_single['color'] = str(_[3])
  
df = pd.concat([df, df_single], axis=0)
df

,height,width,name,color
0,1.151330,0.556295,Cumari,Yellow
1,1.103486,0.609101,Cumari,Yellow
2,0.968693,0.580969,Cumari,Yellow
3,0.950795,0.642170,Cumari,Yellow
4,0.930211,0.627335,Cumari,Yellow
...,...,...,...,...
99995,1.097707,0.741078,Cumari,Yellow
99996,0.916963,0.559599,Cumari,Yellow
99997,0.892156,0.589381,Cumari,Yellow
99998,1.187542,0.547312,Cumari,Yellow


In [8]:
#backup
X = df.copy()

def one_hot(df, partitions):
  #togliamo le colonne da X
  for col in partitions:
    k = df.pop(col)
    k = pd.get_dummies(k, prefix=col)
    df = pd.concat([df, k] , axis=1)
  return df
  
X = one_hot(X, ['color'])
X

,height,width,name,color_Yellow
0,1.151330,0.556295,Cumari,1
1,1.103486,0.609101,Cumari,1
2,0.968693,0.580969,Cumari,1
3,0.950795,0.642170,Cumari,1
4,0.930211,0.627335,Cumari,1
...,...,...,...,...
99995,1.097707,0.741078,Cumari,1
99996,0.916963,0.559599,Cumari,1
99997,0.892156,0.589381,Cumari,1
99998,1.187542,0.547312,Cumari,1


In [9]:
y = X.pop('name')
y

0        Cumari
1        Cumari
2        Cumari
3        Cumari
4        Cumari
          ...  
99995    Cumari
99996    Cumari
99997    Cumari
99998    Cumari
99999    Cumari
Name: name, Length: 100000, dtype: object

In [10]:
X

,height,width,color_Yellow
0,1.151330,0.556295,1
1,1.103486,0.609101,1
2,0.968693,0.580969,1
3,0.950795,0.642170,1
4,0.930211,0.627335,1
...,...,...,...
99995,1.097707,0.741078,1
99996,0.916963,0.559599,1
99997,0.892156,0.589381,1
99998,1.187542,0.547312,1


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [13]:
clf.score(X_test, y_test, sample_weight=None)

1.0